# Silver Layer

## Imports

In [1]:
import pandas as pd
from deep_translator import GoogleTranslator

from os import makedirs
import sqlite3

## Creating Necessary Folder

In [2]:
makedirs("data-lake/silver", exist_ok=True)

## Reading Bronze Layers

In [3]:
bronze_data_users = pd.read_parquet("data-lake/bronze/users.parquet")

In [4]:
bronze_data_todos = pd.read_parquet("data-lake/bronze/todos.parquet")

## Processing Todos Data

In [5]:
translator = GoogleTranslator("latin", "english")

In [6]:
bronze_data_todos["title_translated"] = bronze_data_todos["title"].apply(lambda row: translator.translate(row))

## Processing Users Data

In [7]:
silver_address = pd.json_normalize(bronze_data_users["address"])

In [8]:
silver_address["userId"] = silver_address.index + 1

In [9]:
silver_company = pd.json_normalize(bronze_data_users["company"])

In [10]:
silver_company["userId"] = silver_company.index + 1

In [11]:
silver_users = bronze_data_users.drop(["address", "company"], axis=1)

## Loading Silver Data

In [12]:
conn = sqlite3.connect("data-lake/silver/database.db")

In [13]:
bronze_data_todos.to_sql("todos", conn, if_exists="replace", index=False)

200

In [14]:
silver_users.to_sql("users", conn, if_exists="replace", index=False)

10

In [15]:
silver_address.to_sql("address", conn, if_exists="replace", index=False)

10

In [16]:
silver_company.to_sql("company", conn, if_exists="replace", index=False)

10